In [1]:
#import warnings
#warnings.filterwarnings("ignore")
import os
from glob import glob
import argparse

In [2]:
import dask
from dask_ml import linear_model as dlm
import dask.dataframe as dpd
import joblib
import pandas as pd
from dask.distributed import Client,LocalCluster

In [3]:
n_jobs = 4
tmp_dir =  '/ksozykinraid/tmp/'
dask.config.set({'temporary_directory': tmp_dir})
os.environ['JOBLIB_TEMP_FOLDER'] = tmp_dir 

In [4]:
cluster = LocalCluster(threads_per_worker=2, n_workers=n_jobs,memory_limit='2GB')
client = Client(cluster)

In [5]:
from solvers import XgbSolver,XgbDaskSolver

In [6]:
from solvers import BaseSolver

In [7]:
class SgdDaskSolver(BaseSolver):
    
    def __init__(self,max_years=10,
                      seed = 42,
                      test_size = 0.25,
                      data_dir = '/ksozykinraid/data/nycflights',
                      n_jobs = 4):
        
        self.set_const(data_dir,max_years,seed,test_size,n_jobs)
        self.model = dlm.LinearRegression()
        self.backend = dpd
        self.backend_name = 'dask'
        self.param_grid = {
            'max_iter': [100, 500, 1000]
        }

In [8]:
def cv(self):
    grid_search = GridSearchCV(self.model, self.param_grid,
                               cv=3, n_jobs=1)
    with joblib.parallel_backend("dask"):
        grid_search.fit(self.X_val, self.y_val)
    self.model = self.model.__class__(**grid_search.best_params_,scheduler=client)

In [9]:
#pd_solver = XgbSolver(n_jobs=n_jobs)
#pd_solver.solve()
#pd_solver.info()

In [10]:
dask_solver = SgdDaskSolver(n_jobs=n_jobs)

In [ ]:
dask_solver.solve()

In [ ]:
dask_solver.info()

In [11]:
import dask_ml.model_selection as dcv
from sklearn import svm, datasets
iris = datasets.load_iris()
parameters = {
        'max_iter': [100, 500, 1000],
        
    }
m = dlm.LinearRegression()
clf = dcv.GridSearchCV(m, parameters)

In [12]:
dpd.from_pandas(pd.DataFrame(iris.target),npartitions=2)

,0
npartitions=2,
0,int64
75,...
149,...


In [ ]:
clf.fit(dpd.from_pandas(pd.DataFrame(iris.data),npartitions=2), 
        dpd.from_pandas(pd.DataFrame(iris.target),npartitions=2))

In [ ]:
import dask

In [ ]:
dask.dataframe.from_array(iris.data)

In [ ]:
m